# Montar drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## revisar particion de datos

In [ ]:
import os

dir_project = '/content/drive/MyDrive/projects/brainTumor'

dir_dataset = os.path.join(dir_project,'dataset')

train_path = 'train'
val_path = 'val'

train_dir = os.path.join(dir_dataset, train_path)
val_dir = os.path.join(dir_dataset, val_path)

def count_files_in_subdirectories(base_directory):
    """
    Counts the number of files in each immediate subdirectory
    of a given base directory.
    """
    if not os.path.isdir(base_directory):
        print(f"Error: Directory not found at {base_directory}")
        return

    print(f"\nCounting files in subdirectories of: {base_directory}")
    total_files = 0
    for subdir_name in os.listdir(base_directory):
        subdir_path = os.path.join(base_directory, subdir_name)
        if os.path.isdir(subdir_path): # Ensure it's a directory
            num_files = len([f for f in os.listdir(subdir_path) if os.path.isfile(os.path.join(subdir_path, f))])
            print(f"  {subdir_name}/: {num_files} files")
            total_files += num_files
    print(f"Total files in {base_directory}: {total_files}")

# Contar archivos train
count_files_in_subdirectories(train_dir)

# Contar archivos validation
count_files_in_subdirectories(val_dir)

# Para los directorios se usa la regla 80/20


# Importar librerias

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall, AUC
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import numpy as np
from sklearn.utils import class_weight


# Preparar datos

## Constantes de datos de entrada

In [ ]:
# Constantes para EfficientNetB0

# imagenes
IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 32

# learning_rate
learning_rate = 0.001

# epochs
epochs = 15

# dropout
DROPOUT = 0.5


## aumentar data de entranamiento

In [ ]:
# Data Augmentation para train
train_image_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# preprocessing para val
validation_image_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

train_data_gen = train_image_generator.flow_from_directory(
    batch_size=BATCH_SIZE,
    directory=train_dir,
    shuffle=True,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode='binary' # binario
)

val_data_gen = validation_image_generator.flow_from_directory(
    batch_size=BATCH_SIZE,
    directory=val_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode='binary' # binario
)


## Verificar clases

In [ ]:
print("\n--- Verificación de Clases e Índices ---")

# Acceder al mapeo de class_indices
class_indices = train_data_gen.class_indices
print(f"Mapeo de clases a índices: {class_indices}")

# 2. Obtener los nombres de las clases en el orden de los índices (para predicciones)
# Esto es importante porque el modelo predice un índice (0, 1, 2...)
# y se necesita saber a qué clase corresponde cada índice.
num_classes = len(class_indices)
class_names = [None] * num_classes # Crear una lista vacía con el tamaño correcto

for class_name, index in class_indices.items():
    class_names[index] = class_name

print(f"Nombres de las clases en el orden de los índices del modelo: {class_names}")

## Calcular pesos

In [ ]:
print("\n--- Calculando pesos de las clases para manejar el desbalance ---")
# Obtener los índices de clase de todas las imágenes en el conjunto de entrenamiento
labels = train_data_gen.classes

# Calcular los pesos de las clases
weights = class_weight.compute_class_weight(
    class_weight='balanced', # 'balanced' es la opción clave aquí
    classes=np.unique(labels), # Asegura que todas las clases estén representadas
    y=labels # Las etiquetas de clase de tus datos de entrenamiento
)

# Convertir el array de pesos a un diccionario, que es el formato que espera Keras
class_weights = dict(enumerate(weights))

print(f"Pesos de las clases calculados: {class_weights}")

## Cargar modelo preentrenado EfficientNetB0

In [ ]:
# cargar pre-entrenado EfficientNetB0
base_model = EfficientNetB0(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3),
                            include_top=False,
                            weights='imagenet')

# congelar modelo base
base_model.trainable = False

# cabezera de capas personalizadas
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(DROPOUT)(x)
# funcion de activacion sigmoide
outputs = Dense(1, activation='sigmoid')(x)


## Compilar modelo

In [ ]:
model = Model(inputs=base_model.input, outputs=outputs)

model.compile(optimizer=Adam(learning_rate=learning_rate),
              loss='binary_crossentropy', # binario
              metrics=['accuracy', Precision(name='precision'), Recall(name='recall'), AUC(name='auc')])

model.summary()


## Definir pasos y callbacks

In [ ]:
# Mejor modelo
log_dir = os.path.join(dir_project, "logs")
pathModeloMejorPrecision = os.path.join(dir_project, 'modeloMejorPrecision.h5')

# steps_per_epoch y validation_steps
steps_per_epoch_train = train_data_gen.samples // BATCH_SIZE
if train_data_gen.samples % BATCH_SIZE != 0:
    steps_per_epoch_train += 1

steps_per_epoch_val = val_data_gen.samples // BATCH_SIZE
if val_data_gen.samples % BATCH_SIZE != 0:
    steps_per_epoch_val += 1

# Callbacks para mejorar entrenamiento, principalmente precision
callbacks = [
    # EarlyStopping: Monitorear 'val_precision' y mejores pesos
    EarlyStopping(monitor='val_precision',
                  patience=5, # cuando para
                  mode='max', # 'max' precision
                  restore_best_weights=True,
                  verbose=1),

    # ModelCheckpoint: guarda modelo con mejor 'val_precision'
    ModelCheckpoint(filepath=pathModeloMejorPrecision,
                    monitor='val_precision',
                    save_best_only=True,
                    mode='max', # 'max' precision
                    verbose=1),

    # ReduceLROnPlateau: monitorea y reduce precision
    ReduceLROnPlateau(monitor='val_precision',
                      factor=0.2, # Reducir learning rate 20%
                      patience=3, # cuando para
                      mode='max',
                      min_lr=0.00001, # learning rate minimo
                      verbose=1),
]


# Entrenamiento

## Entrenar modelo base EfficientNetB0

### Modelo base

In [ ]:
print("Iniciando entrenamiento...")
# cambiar nombre history_base
history_base = model.fit(
    train_data_gen,
    # Pasos por epoca train
    steps_per_epoch=steps_per_epoch_train,
    epochs=epochs,
    validation_data=val_data_gen,
    # Pasos por epoca validación
    validation_steps=steps_per_epoch_val,
    class_weight=class_weights, # pesos por clase para evitar desbalance
    callbacks=callbacks # callbacks optimiza entrenamiento
)
print("Entrenamiento finalizado.")


### Cargar modelo base

In [ ]:
# Cargar el mejor modelo guardado de la fase de extracción de características
try:
    model = tf.keras.models.load_model(pathModeloMejorPrecision)
    print(f"Mejor modelo de extracción de características cargado desde: {pathModeloMejorPrecision}")
except Exception as e:
    print(f"No se pudo cargar el mejor modelo de extracción de características. Continuando con el modelo actual. Error: {e}")


## Entrenamiento Fine-Tuning

### Configurar parametros Fine-Tuning

In [ ]:
# 1. Descongelar el modelo base
base_model.trainable = True

# 2. Determinar el punto de corte
# EfficientNetB0 tiene ~237 capas, descongelar a partir de la capa 200
# para ajustar los últimos bloques de extracción de características de alto nivel.
FINE_TUNE_AT_LAYER = 200

# Congelar todas las capas antes del punto de corte
for layer in base_model.layers[:FINE_TUNE_AT_LAYER]:
    layer.trainable = False

# Mantener las capas de BatchNormalization en modo inferencia
# Esto es vital en EfficientNet para no desestabilizar el entrenamiento.
for layer in base_model.layers:
    if isinstance(layer, tf.keras.layers.BatchNormalization):
        layer.trainable = False

print(f"Capas totales en el modelo base: {len(base_model.layers)}")
print(f"Capas entrenables tras descongelar: {len(model.trainable_variables)}")

# 3. Re-compilar con una tasa de aprendizaje extremadamente baja
fine_tune_learning_rate = 1e-5 # 0.00001

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=fine_tune_learning_rate, clipnorm=1.0),
    loss='binary_crossentropy', # Riguroso para 2 clases
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.AUC(name='auc')
    ]
)


### Callbacks para Fine-Tuning

In [ ]:
epochs_fine_tuning = 10
# Recuperar última época de la fase de entrenamiento base
initial_epoch_fine_tune = len(history_base.history['accuracy'])

checkpoint_filepath_ft = os.path.join(dir_project, 'modeloFineTuneBrainTumor.h5')

callbacks_ft = [
    tf.keras.callbacks.EarlyStopping(monitor='val_precision', patience=5, mode='max', restore_best_weights=True),
    tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath_ft, monitor='val_precision', save_best_only=True, mode='max'),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_precision', factor=0.2, patience=2, min_lr=1e-7)
]


### Entrenamiento Fine-Tuning

In [ ]:
print(f"Iniciando ajuste fino desde la época {initial_epoch_fine_tune}...")

history_fine_tuning = model.fit(
    train_data_gen,
    steps_per_epoch=steps_per_epoch_train,
    epochs=initial_epoch_fine_tune + epochs_fine_tuning,
    initial_epoch=initial_epoch_fine_tune,
    validation_data=val_data_gen,
    validation_steps=steps_per_epoch_val,
    callbacks=callbacks_ft
)


### Cargar modelo tras Fine Tuning

In [ ]:
# Cargar el mejor modelo guardado de la fase de ajuste fino (este será el modelo final)
try:
    final_model = tf.keras.models.load_model(checkpoint_filepath_ft)
    print(f"Modelo final cargado (mejor de ajuste fino): {checkpoint_filepath_ft}")
except Exception as e:
    print(f"No se pudo cargar el mejor modelo de ajuste fino. Usando el modelo actual. Error: {e}")
    final_model = model # Si falla, usa el modelo tal como quedó al final del entrenamiento


# Guardar modelo

In [ ]:
# Guardar el modelo entrenado
# antes h5, luego keras por integracion con fastapi
path_final_keras_model = os.path.join(dir_project, 'modeloBrainTumor.keras')
final_model.save(path_final_keras_model)
print(f"Modelo Keras final guardado exitosamente en: {path_final_keras_model}")


# Metricas y Evaluacion

## Librerias de evaluacion



In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


## Cargar modelo y generar predicciones

In [ ]:
# Cargar el mejor modelo obtenido
final_model = tf.keras.models.load_model(checkpoint_filepath_ft)

# Generar predicciones
val_data_gen.reset()
y_true = val_data_gen.classes
# Predicciones probabilísticas
preds = final_model.predict(val_data_gen, verbose=1)
# Convertir a clases binarias (0 o 1) usando umbral 0.5
y_pred = (preds > 0.5).astype(int).ravel()

class_names = list(val_data_gen.class_indices.keys())


## Informe de clasificacion

In [ ]:
# --- Informe de Clasificación ---
print("\n" + "="*40)
print("INFORME DE CLASIFICACIÓN FINAL")
print("="*40)
print(classification_report(y_true, y_pred, target_names=class_names))


## Matriz de confusion

In [ ]:
# --- Matriz de Confusión ---
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Reds', xticklabels=class_names, yticklabels=class_names)
plt.title('Matriz de Confusión: Detección de Tumor')
plt.xlabel('Predicción del Modelo')
plt.ylabel('Realidad (Gold Standard)')
plt.show()


## Graficas de rendimiento

In [ ]:
def plot_full_history(h1, h2):
    acc = h1.history['accuracy'] + h2.history['accuracy']
    val_acc = h1.history['val_accuracy'] + h2.history['val_accuracy']
    loss = h1.history['loss'] + h2.history['loss']
    val_loss = h1.history['val_loss'] + h2.history['val_loss']

    plt.figure(figsize=(12, 5))

    # Gráfica de Accuracy
    plt.subplot(1, 2, 1)
    plt.plot(acc, label='Entrenamiento')
    plt.plot(val_acc, label='Validación')
    plt.axvline(x=len(h1.history['accuracy'])-1, color='r', linestyle='--', label='Inicio Fine-tuning')
    plt.title('Precisión (Accuracy)')
    plt.legend()

    # Gráfica de Loss
    plt.subplot(1, 2, 2)
    plt.plot(loss, label='Entrenamiento')
    plt.plot(val_loss, label='Validación')
    plt.axvline(x=len(h1.history['loss'])-1, color='r', linestyle='--', label='Inicio Fine-tuning')
    plt.title('Pérdida (Loss)')
    plt.legend()
    plt.show()

plot_full_history(history_base, history_fine_tuning)
